In [1]:
# Set paths
idmap_train_dir = '../../data/input/trainmap.csv'
methy_train_dir = '../../data/input/traindata.csv'
idmap_test_dir = '../../data/input/testmap.csv'
methy_test_dir = '../../data/input/testdata.csv'
output_dir = "../../data/output/"

In [2]:
import numpy as np
import pandas as pd
import h5py

# Load idmap data from .csv file
def load_idmap_csv(file):
    sample_type_mapping = {'control': 0, 'disease tissue': 1}
    
    idmap = pd.read_csv(file, sep=',')
    age = idmap.age.to_numpy()
    age = age.astype(np.float32)
    sample_type = idmap.sample_type.replace(sample_type_mapping)
    
    return age, sample_type

# Load methylation data from .h5 file. 
def load_methylation_h5(file, rows=None):
    methylation = h5py.File(file, 'r')['data']
    h5py.File(file, 'r').close()

    if rows != None:
        return methylation[:, :rows]
    else:
        return methylation[:, :]

In [3]:
# Load idmap
age, sample_type = load_idmap_csv(idmap_train_dir)
print("ID map loaded.")

age_train = age[:5000]
age_test = age[5000:]

print("age_train shape:", age_train.shape)
print("age_test shape:",age_test.shape)

sample_train = sample_type[:5000]
sample_test = sample_type[5000:]

print("sample_train shape:", sample_train.shape)
print("sample_test shape:",sample_test.shape)

# Load methylation data
methylation_data = load_methylation_h5('../../data/input/train.h5', rows=20000)
print("\nMethylation data loaded.")

train_data = methylation_data[:5000]
test_data = methylation_data[5000:]

print("methylation_train shape:", train_data.shape)
print("methylation_test shape:", test_data.shape)

ID map loaded.
age_train shape: (5000,)
age_test shape: (3233,)
sample_train shape: (5000,)
sample_test shape: (3233,)

Methylation data loaded.
methylation_train shape: (5000, 20000)
methylation_test shape: (3233, 20000)


In [4]:
from sklearn.model_selection import train_test_split
import random

random.seed(42)

# Split data into training and validation sets
indices = np.arange(sample_train.shape[0])
[indices_train, indices_valid, age_train, age_valid] = train_test_split(indices, age_train, test_size=0.3, shuffle=True)

methylation_train = train_data[indices_train]
methylation_valid = train_data[indices_valid]

sample_type_train = sample_train[indices_train]
sample_type_valid = sample_train[indices_valid]

feature_size = methylation_train.shape[1]

In [13]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Create RFE model
def train_selector(X_train, y_train, num_features):
    model = LinearRegression()
    selector = RFE(model, n_features_to_select=num_features)
    selector = selector.fit(X_train, y_train)

    return model, selector


# Train model
def train_model(X_train, y_train, model):
    model.fit(X_train, y_train)
    return model


# Evaluate model
def evaluate_ml(y_true, y_pred, sample_type):
    mae_control = np.mean(
        np.abs(y_true[sample_type == 0] - y_pred[sample_type == 0]))

    case_true = y_true[sample_type == 1]
    case_pred = y_pred[sample_type == 1]
    above = np.where(case_pred >= case_true)
    below = np.where(case_pred < case_true)

    ae_above = np.sum(np.abs(case_true[above] - case_pred[above])) / 2
    ae_below = np.sum(np.abs(case_true[below] - case_pred[below]))
    mae_case = (ae_above + ae_below) / len(case_true)

    mae = np.mean([mae_control, mae_case])
    return mae, mae_control, mae_case


In [15]:
# Train selector
model, selector = train_selector(methylation_train, age_train, 19800)

KeyboardInterrupt: 

In [ ]:
import pickle

# Save selected features to file
with open('../../data/output/selected_features.pkl', 'wb') as file:
    pickle.dump(selector.support_, file)

In [ ]:
# Train model
pred_model = train_model(selector.transform(methylation_train), age_train, model)

In [ ]:
# Evaluate model
age_test_pred = pred_model.predict(test_data)
age_test_eval = evaluate_ml(age_test, age_test_pred, sample_test)
print(f'Test MAE: {age_test_eval}')